# Create ASE database for transition state calculations

- 13 surfaces considered (the most stable surface of Ag, Au, Cd, Cu, Co, Ir, Ni, Os, Pd, Pt, Rh, Ru, Zn)
- Adsorbates contain C, H, O.

## Imports

In [1]:
import os, sys
sys.path.insert(0, '../src')
from subprocess import Popen, PIPE

from ase.db import connect
from ase.io.vasp import read_vasp
from ase.atoms import Atoms
from ase.formula import Formula
from pymatgen.core.composition import Composition

## Global variables

In [2]:
PATH = '/WAREHOUSE/FATS/database'
METAL_PATH = '/WAREHOUSE/GNN/DFT_data/surfaces'
SURFACES = ["ag111", "au111", "cd0001", "co0001", "cu111", "ir111", "ni111", "os0001", "pd111", "pt111", "rh111", "ru0001", "zn0001"]
SUBSETS = [os.path.join(PATH, subset) for subset in os.listdir(PATH)]
SUBSETS.remove(os.path.join(PATH, 'EXTRAS'))  # EXTRAS does not contain TS
ASE_DB_NAME = 'ts.db'

## Auxiliary functions

In [5]:
def get_fragment_energy(structure: dict) -> float:
    """Calculate fragment energy from closed shell structures.
    This function allows to calculate the energy of both open- and closed-shell structures, 
    keeping the same reference.
    Args:
        structure (list[int]): list of atom numbers in the order C, H, O, N, S
    Returns:
        e_fragment (float): fragment energy in eV
    """ 
    # Reference DFT energy for C, H, O, N, S
    e_H2O = -14.21877278  # O
    e_H2 = -6.76639487    # H
    e_NH3 = -19.54236910  # N
    e_H2S = -11.20113092  # S
    e_CO2 = -22.96215586  # C
    # Count elemens in the structure
    n_C = structure["C"]
    n_H = structure["H"]
    n_O = structure["O"]
    n_N = structure["N"]
    n_S = structure["S"]
    return n_C * e_CO2 + (n_O - 2*n_C) * e_H2O + (4*n_C + n_H - 2*n_O - 3*n_N - 2*n_S) * e_H2 * 0.5 + (n_N * e_NH3) + (n_S * e_H2S)

def get_metal(atoms_obj: Atoms) -> str:
    """Get the metal the slab is built of from the output
    Args:
        atoms_obj (Atoms): ASE Atoms object 
    """
    if Composition(str(atoms_obj.symbols)).contains_element_type("metal"):
        for i in range(len(atoms_obj.symbols)):
            if Composition(str(atoms_obj[i].symbol)).contains_element_type("metal"):
                metal = atoms_obj[i].symbol
    else:
        metal = "N/A"
    return metal

def get_no_of_atom_species(readincontcar: Atoms) -> dict:
    """Return number of atom species in a structure
    Args:
        readincontcar (Atoms): Atoms object read in from CONTCAR, OUTCAR, or vasprun.xml
    Returns:
        dict: returns a dict with atom symbol as key and its occurence as value
    """
    formula = str(readincontcar.symbols)  # retrieve chemical formula from Atoms-object
    formula = Formula(formula).format(
        "metal"
    )  # alphabetically ordered with metal first
    elements = ["C", "H", "O", "N", "S"]  # elements of interest
    formula = Formula(formula).format("metal")
    metal = get_metal(readincontcar)
    formula = Formula(formula).count()  # return dictionary; e.g. {'C': 1, 'O': 2}
    no_metal = readincontcar.get_global_number_of_atoms()
    # expand dict by elements with zero value which are not present in formula
    for i in range(len(elements)):
        if elements[i] not in formula:
            formula[elements[i]] = 0
        elif elements[i] in formula:
            no_metal -= formula[elements[i]]
            formula[metal] = no_metal
        else:
            formula[metal] = no_metal
    return formula

def surface_facet(metal: str):
    if metal.capitalize() in ("Ag", "Au", "Cu", "Ir", "Ni", "Pd", "Pt", "Rh"):
        return "fcc(111)"
    elif metal.capitalize() in ("Cd", "Co", "Os", "Ru", "Zn"):
        return "hcp(0001)"
    elif metal.capitalize() in ("Fe"):
        return "bcc(110)"   
    else:
        return "Unknown"

slab_energy_dict = {}
for surface in SURFACES:  
    ps1 = Popen(['grep', 'energy  w', f"{METAL_PATH}/{surface}/OUTCAR"], stdout=PIPE)
    ps2 = Popen(['tail', '-1'], stdin=ps1.stdout, stdout=PIPE)
    ps3 = Popen(['awk', '{print $NF}'], stdin=ps2.stdout, stdout=PIPE)
    energy = float(ps3.communicate()[0].decode("utf-8"))
    slab_energy_dict[surface] = energy

slab_energy_dict["cu111"] = -125.91841662
slab_energy_dict["ru0001"] = -317.00141469
print(slab_energy_dict)

{'ag111': -125.30436866, 'au111': -150.76413218, 'cd0001': -36.05766447, 'co0001': -327.6932973, 'cu111': -125.91841662, 'ir111': -408.05800262, 'ni111': -250.91869356, 'os0001': -518.61939974, 'pd111': -241.23300654, 'pt111': -282.75561201, 'rh111': -333.72858462, 'ru0001': -317.00141469, 'zn0001': -53.85841374}


In [6]:
def is_vasp_calc(path: str):
    """
    Detect if the directory contains a VASP simulation.
    """
    file_names = ["INCAR", "OUTCAR", "CONTCAR"]
    # define condition that says if the directory contains a VASP simulation
    if os.path.isdir(path):
        if all([os.path.exists(os.path.join(path, file)) for file in file_names]):
            return True
        else:
            return False
    else:
        return False

## Select only TS simulations in the database directory

In [7]:
CALCS_LIST = []
for subset in SUBSETS:
    for root, dirs, _ in os.walk(subset):
        for d in dirs:   
            calc_path = os.path.join(root, d)            
            if is_vasp_calc(calc_path) and "frq" not in calc_path and "old" not in calc_path:
                CALCS_LIST.append(calc_path)
            else:
                continue
print(f"Number of VASP TST calculations: {len(CALCS_LIST)}")

Number of VASP TST calculations: 2940


## Create database

In [8]:
with connect(os.path.join(".", ASE_DB_NAME)) as db:
    for calc in CALCS_LIST:
        calc_type = "transition_state"
        try:
            atoms_object = read_vasp(os.path.join(calc, "CONTCAR"))
        except:
            continue
        try:
            ps1 = Popen(['grep', 'energy  w', str(os.path.join(calc, "OUTCAR"))], stdout=PIPE)
            ps2 = Popen(['tail', '-1'], stdin=ps1.stdout, stdout=PIPE)
            ps3 = Popen(['awk', '{print $NF}'], stdin=ps2.stdout, stdout=PIPE)
            energy = float(ps3.communicate()[0].decode("utf-8"))
        except:
            continue
        metal = atoms_object.get_chemical_formula(mode="metal")[:2]
        surf_facet = surface_facet(metal)
        slab_energy = slab_energy_dict[metal.lower()+surf_facet[4:].replace(")", "")]
        scaled_energy = energy - slab_energy
        num_atoms_dict = get_no_of_atom_species(atoms_object)
        nC = num_atoms_dict["C"]
        nH = num_atoms_dict["H"]
        nO = num_atoms_dict["O"]
        nN = num_atoms_dict["N"]
        nS = num_atoms_dict["S"]
        db.write(atoms_object,
                    calc_type=calc_type,
                    metal=metal,
                    facet=surf_facet,
                    e_tot=energy,
                    e_slab=slab_energy,
                    scaled_energy=scaled_energy,
                    n_C=nC,
                    n_H=nH,
                    n_O=nO,
                    n_N=nN,
                    n_S=nS
                    )
        print("Added to database: ", calc)          

Added to database:  /WAREHOUSE/FATS/database/rxn-pt/set-xx2x-wg/pt-0211+0011-0111+0111
Added to database:  /WAREHOUSE/FATS/database/rxn-pt/set-xx2x-wg/pt-1122+0011-1021+0111
Added to database:  /WAREHOUSE/FATS/database/rxn-pt/set-xx2x-wg/pt-1021+0000-1011+0011
Added to database:  /WAREHOUSE/FATS/database/rxn-pt/set-xx2x-wg/pt-0021+0000-0011+0011
Added to database:  /WAREHOUSE/FATS/database/rxn-pt/set-xx2x-wg/pt-1122+0000-1021+0101
Added to database:  /WAREHOUSE/FATS/database/rxn-pt/set-xx2x-wg/pt-1122+0111-1021+0211
Added to database:  /WAREHOUSE/FATS/database/rxn-pt/set-xx2x-wg/pt-1122+0000-1011+0111
Added to database:  /WAREHOUSE/FATS/database/rxn-pt/set-2xxx-oc/pt-2323-2212+0111
Added to database:  /WAREHOUSE/FATS/database/rxn-pt/set-2xxx-oc/pt-2423-2415+0011
Added to database:  /WAREHOUSE/FATS/database/rxn-pt/set-2xxx-oc/pt-2521-2412+0111
Added to database:  /WAREHOUSE/FATS/database/rxn-pt/set-2xxx-oc/pt-2621-2513+0111
Added to database:  /WAREHOUSE/FATS/database/rxn-pt/set-2xxx-oc